In [1]:
#connect google drive colab notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import directory from google drive
import os
os.chdir('/content/drive/MyDrive/parquet')

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Flatten
from tensorflow.keras.models import Model

In [5]:
import glob
# Define the path to the directory containing the Parquet files
directory_path = "/content/drive/MyDrive/parquet/train/"

# Use glob to find all Parquet files in the directory
file_paths = glob.glob(directory_path + "*.parquet")

# Read the Parquet files into a Pandas dataframe
train_df = pd.concat([pd.read_parquet(f) for f in file_paths], ignore_index=True)

In [6]:
train_df

,product_id,title,description,tags,type,room,craft_type,recipient,material,occasion,...,art_subject,style,shape,pattern,bottom_category_id,bottom_category_text,top_category_id,top_category_text,color_id,color_text
0,1367151543,Quilted pie/casserole carrier,"Ingenious and pretty 18” square carrier, made ...","brown,red,casserole,pie carrier,insulated,quil...",physical,None,None,None,None,None,...,None,None,None,None,154,bags_and_purses.food_and_insulated_bags.picnic...,2,bags_and_purses,4,brown
1,702267069,Vintage Red Gingham Wicker Suitcase Picnic Bas...,This fun Woven Wicker Basket is perfect for a ...,"picnic basket,wicker suitcase,picnic suitcase,...",physical,None,None,None,None,None,...,None,None,None,None,154,bags_and_purses.food_and_insulated_bags.picnic...,2,bags_and_purses,4,brown
2,1186742238,Picnic gift set - Longtail Tit design - lunch ...,A co-ordinating gift set - A perfect gift for ...,"eco gift,gift for outdoors,nature gift,eco sna...",physical,None,None,None,None,None,...,None,None,None,None,154,bags_and_purses.food_and_insulated_bags.picnic...,2,bags_and_purses,9,green
3,249113317,Reusable linen bread bag Round loaf Linen brea...,Natural linen bread bag is ideal for carrying ...,"Linen bread bag,natural linen bags,Bread bag,S...",physical,None,None,women,None,housewarming,...,None,traditional,None,None,154,bags_and_purses.food_and_insulated_bags.picnic...,2,bags_and_purses,19,other
4,700470885,"Picnic basket, small. Including plates and cup...",Picnic basket including covered for two: Plate...,None,physical,None,None,None,None,None,...,None,None,None,None,154,bags_and_purses.food_and_insulated_bags.picnic...,2,bags_and_purses,0,beige
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245480,1372093313,"Latina Gifts, Latina Own Business, Latina Cand...",You won’t believe how amazing this soy blend c...,"Gifts Best Friend,Positive Thinking,latina can...",physical,bedroom,None,None,None,bachelorette party,...,None,None,None,None,1009,home_and_living.home_decor.candles_and_holders...,8,home_and_living,17,white
245481,1389863144,6.5oz Smells Like Seaglass and Ambre Diffuser,SCENT: SEAGLASS AND AMBRE\n\nREEDS WILL ABSORB...,"Clearance,COCO by Stone,Diffuser,Seaglass Ambr...",physical,None,None,None,None,1st birthday,...,None,None,None,None,1009,home_and_living.home_decor.candles_and_holders...,8,home_and_living,0,beige
245482,1400071683,Bookish Gifts | Reading Candle | Scented Soy C...,"Packed with immersive aromas, these scented ca...","reading candle,acotar candle,book candle,my re...",physical,None,None,None,None,None,...,None,None,None,None,1009,home_and_living.home_decor.candles_and_holders...,8,home_and_living,15,rose gold
245483,1294408254,Baptismal candle 40 x 4 cm/communion candle/go...,"Welcome to Kerzenglanz.by.Romy,\nWhoever finds...",None,physical,nursery,None,None,None,first communion,...,None,None,None,None,1009,home_and_living.home_decor.candles_and_holders...,8,home_and_living,17,white


In [8]:
train_df['title']=train_df['title'].fillna('none')
train_df['description']=train_df['description'].fillna('none')

train_df['tags']=train_df['tags'].fillna('none')

In [9]:
train_df

,product_id,title,description,tags,type,room,craft_type,recipient,material,occasion,...,art_subject,style,shape,pattern,bottom_category_id,bottom_category_text,top_category_id,top_category_text,color_id,color_text
0,1367151543,Quilted pie/casserole carrier,"Ingenious and pretty 18” square carrier, made ...","brown,red,casserole,pie carrier,insulated,quil...",physical,None,None,None,None,None,...,None,None,None,None,154,bags_and_purses.food_and_insulated_bags.picnic...,2,bags_and_purses,4,brown
1,702267069,Vintage Red Gingham Wicker Suitcase Picnic Bas...,This fun Woven Wicker Basket is perfect for a ...,"picnic basket,wicker suitcase,picnic suitcase,...",physical,None,None,None,None,None,...,None,None,None,None,154,bags_and_purses.food_and_insulated_bags.picnic...,2,bags_and_purses,4,brown
2,1186742238,Picnic gift set - Longtail Tit design - lunch ...,A co-ordinating gift set - A perfect gift for ...,"eco gift,gift for outdoors,nature gift,eco sna...",physical,None,None,None,None,None,...,None,None,None,None,154,bags_and_purses.food_and_insulated_bags.picnic...,2,bags_and_purses,9,green
3,249113317,Reusable linen bread bag Round loaf Linen brea...,Natural linen bread bag is ideal for carrying ...,"Linen bread bag,natural linen bags,Bread bag,S...",physical,None,None,women,None,housewarming,...,None,traditional,None,None,154,bags_and_purses.food_and_insulated_bags.picnic...,2,bags_and_purses,19,other
4,700470885,"Picnic basket, small. Including plates and cup...",Picnic basket including covered for two: Plate...,none,physical,None,None,None,None,None,...,None,None,None,None,154,bags_and_purses.food_and_insulated_bags.picnic...,2,bags_and_purses,0,beige
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245480,1372093313,"Latina Gifts, Latina Own Business, Latina Cand...",You won’t believe how amazing this soy blend c...,"Gifts Best Friend,Positive Thinking,latina can...",physical,bedroom,None,None,None,bachelorette party,...,None,None,None,None,1009,home_and_living.home_decor.candles_and_holders...,8,home_and_living,17,white
245481,1389863144,6.5oz Smells Like Seaglass and Ambre Diffuser,SCENT: SEAGLASS AND AMBRE\n\nREEDS WILL ABSORB...,"Clearance,COCO by Stone,Diffuser,Seaglass Ambr...",physical,None,None,None,None,1st birthday,...,None,None,None,None,1009,home_and_living.home_decor.candles_and_holders...,8,home_and_living,0,beige
245482,1400071683,Bookish Gifts | Reading Candle | Scented Soy C...,"Packed with immersive aromas, these scented ca...","reading candle,acotar candle,book candle,my re...",physical,None,None,None,None,None,...,None,None,None,None,1009,home_and_living.home_decor.candles_and_holders...,8,home_and_living,15,rose gold
245483,1294408254,Baptismal candle 40 x 4 cm/communion candle/go...,"Welcome to Kerzenglanz.by.Romy,\nWhoever finds...",none,physical,nursery,None,None,None,first communion,...,None,None,None,None,1009,home_and_living.home_decor.candles_and_holders...,8,home_and_living,17,white


In [10]:
# Preprocess the data
# Clean and normalize text data
train_df["title"] = train_df["title"].str.lower().replace("[^a-zA-Z0-9]", " ", regex=True)
train_df["description"] = train_df["description"].str.lower().replace("[^a-zA-Z0-9]", " ", regex=True)
train_df["tags"] = train_df["tags"].str.lower().replace("[^a-zA-Z0-9]", " ", regex=True)


In [11]:
train_df.title

0                             quilted pie casserole carrier
1         vintage red gingham wicker suitcase picnic bas...
2         picnic gift set   longtail tit design   lunch ...
3         reusable linen bread bag round loaf linen brea...
4         picnic basket  small  including plates and cup...
                                ...                        
245480    latina gifts  latina own business  latina cand...
245481        6 5oz smells like seaglass and ambre diffuser
245482    bookish gifts   reading candle   scented soy c...
245483    baptismal candle 40 x 4 cm communion candle go...
245484    aesthetic lantern candle  lamp candle  unique ...
Name: title, Length: 245485, dtype: object

In [12]:
from sklearn.model_selection import train_test_split
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_df[["title", "description", "tags", "room", "craft_type", "recipient", "material", "occasion", "holiday", "art_subject", "style", "shape", "pattern"]], 
                                                  train_df[["top_category_id", "bottom_category_id", "color_id"]], 
                                                  test_size=0.2, 
                                                  random_state=42)

In [13]:
X_train["title"]

104891    drawstring ruched women pencil midi skirt slit...
79990     ivory baby romper  baby boy romper  off white ...
242791          3 panel leather folding screen room divider
188747    personalised nipple clamp or clit tweezers cus...
84017                       all white custom bling converse
                                ...                        
119879                                  marbled ceramic diz
103694    beach home artwork  encaustic painting  coasta...
131932    penguin spacebar keycap  custom spacebar keyca...
146867    designer black wedding lehenga choli for women...
121958    kids footwear toddler shoes boys girls kids sh...
Name: title, Length: 196388, dtype: object

In [14]:
# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train["title"])
X_train_title = pad_sequences(tokenizer.texts_to_sequences(X_train["title"]), maxlen=100)
X_val_title = pad_sequences(tokenizer.texts_to_sequences(X_val["title"]), maxlen=100)
tokenizer.fit_on_texts(X_train["description"])
X_train_desc = pad_sequences(tokenizer.texts_to_sequences(X_train["description"]), maxlen=100)
X_val_desc = pad_sequences(tokenizer.texts_to_sequences(X_val["description"]), maxlen=100)
tokenizer.fit_on_texts(X_train["tags"])
X_train_tags = pad_sequences(tokenizer.texts_to_sequences(X_train["tags"]), maxlen=100)
X_val_tags = pad_sequences(tokenizer.texts_to_sequences(X_val["tags"]), maxlen=100)

In [15]:
X_train=X_train.fillna(value='a')
X_val=X_val.fillna(value='b')
y_train=y_train.fillna(value='a')
y_val=y_val.fillna(value='b')

In [16]:

# Define the neural network model
input_title = Input(shape=(100,))
input_desc = Input(shape=(100,))
input_tags = Input(shape=(100,))
input_room = Input(shape=(100,))
input_craft_type = Input(shape=(100,))
input_recipient = Input(shape=(100,))
input_material = Input(shape=(100,))
input_occasion = Input(shape=(100,))
input_holiday = Input(shape=(100,))
input_art_subject = Input(shape=(100,))
input_style = Input(shape=(100,))
input_shape = Input(shape=(100,))
input_pattern = Input(shape=(100,))

In [17]:
input_style

<KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'input_11')>

In [18]:
embed_title = Embedding(5000, 100, input_length=100)(input_title)
embed_desc = Embedding(5000, 100, input_length=100)(input_desc)
embed_tags = Embedding(5000, 100, input_length=100)(input_tags)
embed_room = Embedding(5000, 100, input_length=100)(input_room)
embed_craft_type = Embedding(5000, 100, input_length=100)(input_craft_type)
embed_recipient = Embedding(5000, 100, input_length=100)(input_recipient)
embed_material = Embedding(5000, 100, input_length=100)(input_material)
embed_occassion = Embedding(5000, 100, input_length=100)(input_occasion)
embed_holiday = Embedding(5000, 100, input_length=100)(input_holiday)
embed_art_subject = Embedding(5000, 100, input_length=100)(input_art_subject)
embed_style = Embedding(5000, 100, input_length=100)(input_style)
embed_shape = Embedding(5000, 100, input_length=100)(input_shape)
embed_pattern = Embedding(5000, 100, input_length=100)(input_pattern)

In [19]:
lstm_title = LSTM(64)(embed_title)
lstm_desc = LSTM(64)(embed_desc)
lstm_tags = LSTM(64)(embed_tags)
lstm_room = LSTM(64)(embed_room)
lstm_craft_type = LSTM(64)(embed_craft_type)
lstm_receipient = LSTM(64)(embed_recipient)
lstm_material = LSTM(64)(embed_material)
lstm_occassion = LSTM(64)(embed_occassion)
lstm_holiday = LSTM(64)(embed_holiday)
lstm_art_subject = LSTM(64)(embed_art_subject)
lstm_style = LSTM(64)(embed_style)
lstm_shape = LSTM(64)(embed_shape)
lstm_pattern = LSTM(64)(embed_pattern)

In [20]:
#Merge all inputs
merged_inputs = tf.concat([lstm_title, lstm_desc, lstm_tags], axis=1)

In [21]:
merged_inputs

<KerasTensor: shape=(None, 192) dtype=float32 (created by layer 'tf.concat')>

In [22]:
#Output layers
output_top_cat = Dense(128, activation='relu')(merged_inputs)
output_top_cat = Dense(64, activation='relu')(output_top_cat)
output_top_cat = Dense(32, activation='relu')(output_top_cat)
output_top_cat = Dense(1, activation='sigmoid', name='output_top_cat')(output_top_cat)

output_bottom_cat = Dense(128, activation='relu')(merged_inputs)
output_bottom_cat = Dense(64, activation='relu')(output_bottom_cat)
output_bottom_cat = Dense(32, activation='relu')(output_bottom_cat)
output_bottom_cat = Dense(1, activation='sigmoid', name='output_bottom_cat')(output_bottom_cat)

output_color = Dense(128, activation='relu')(merged_inputs)
output_color = Dense(64, activation='relu')(output_color)
output_color = Dense(32, activation='relu')(output_color)
output_color = Dense(1, activation='sigmoid', name='output_color')(output_color)

In [23]:
#Define the model
model = Model(inputs=[input_title, input_desc, input_tags],
outputs=[output_top_cat, output_bottom_cat, output_color])

In [24]:
#Compile the model
model.compile(optimizer='adam',
loss='binary_crossentropy',
metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [25]:
X_train["room"].astype('category').cat.codes.astype('int32').values

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [26]:
X_train["room"]=X_train["room"].astype('category').cat.codes.astype('int32')
X_train["craft_type"]=X_train["craft_type"].astype('category').cat.codes.astype('int32')
X_train["recipient"]=X_train["recipient"].astype('category').cat.codes.astype('int32')
X_train["material"]=X_train["material"].astype('category').cat.codes.astype('int32')
X_train["occasion"]=X_train["occasion"].astype('category').cat.codes.astype('int32')
X_train["holiday"]=X_train["holiday"].astype('category').cat.codes.astype('int32')
X_train["art_subject"]=X_train["art_subject"].astype('category').cat.codes.astype('int32')
X_train["style"]=X_train["style"].astype('category').cat.codes.astype('int32')
X_train["shape"]=X_train["shape"].astype('category').cat.codes.astype('int32')
X_train["pattern"]=X_train["pattern"].astype('category').cat.codes.astype('int32')

In [27]:
np.array(X_train["room"].values).shape

(196388,)

In [28]:
#Train the model
model.fit([X_train_title, X_train_desc, X_train_tags],
[y_train["top_category_id"], y_train["bottom_category_id"], y_train["color_id"]],
validation_data=([X_val_title, X_val_desc, X_val_tags],
[y_val["top_category_id"], y_val["bottom_category_id"], y_val["color_id"]]),
batch_size=32,
epochs=1,
callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)])


6138/6138 [==============================] - 1571s 254ms/step - loss: -65028749262848.0000 - output_top_cat_loss: -141536067584.0000 - output_bottom_cat_loss: -64730395836416.0000 - output_color_loss: -156778774528.0000 - output_top_cat_binary_accuracy: 0.0354 - output_top_cat_precision: 0.9488 - output_top_cat_recall: 1.0000 - output_bottom_cat_binary_accuracy: 3.7681e-04 - output_bottom_cat_precision: 1.0000 - output_bottom_cat_recall: 0.9999 - output_color_binary_accuracy: 0.1321 - output_color_precision: 0.9485 - output_color_recall: 0.9999 - val_loss: -272517419761664.0000 - val_output_top_cat_loss: -591195144192.0000 - val_output_bottom_cat_loss: -271273439526912.0000 - val_output_color_loss: -652770934784.0000 - val_output_top_cat_binary_accuracy: 0.0365 - val_output_top_cat_precision: 0.9480 - val_output_top_cat_recall: 1.0000 - val_output_bottom_cat_binary_accuracy: 2.6478e-04 - val_output_bottom_cat_precision: 1.0000 - val_output_bottom_cat_recall: 1.0000 - val_output_color_b

In [29]:
# Define the path to the directory containing the Parquet files
directory_path = "/content/drive/MyDrive/parquet/test/"

# Use glob to find all Parquet files in the directory
file_paths = glob.glob(directory_path + "*.parquet")

# Read the Parquet files into a Pandas dataframe
test_df = pd.concat([pd.read_parquet(f) for f in file_paths], ignore_index=True)
test_df['title']=test_df['title'].fillna('none')
test_df['description']=test_df['description'].fillna('none')

test_df['tags']=test_df['tags'].fillna('none')

test_df["title"] = test_df["title"].str.lower().replace("[^a-zA-Z0-9]", " ", regex=True)
test_df["description"] = test_df["description"].str.lower().replace("[^a-zA-Z0-9]", " ", regex=True)
test_df["tags"] = test_df["tags"].str.lower().replace("[^a-zA-Z0-9]", " ", regex=True)
y_test_val=y_val
test_df_title = pad_sequences(tokenizer.texts_to_sequences(test_df["title"]), maxlen=100)
test_df_desc = pad_sequences(tokenizer.texts_to_sequences(test_df["description"]), maxlen=100)
test_df_tags = pad_sequences(tokenizer.texts_to_sequences(test_df["tags"]), maxlen=100)

In [30]:
test_df.head(2).to_csv("new.csv")

In [31]:
f1_top_cat = f1_score(y_test_val["top_category_id"], y_val["top_category_id"], average="micro")
f1_bottom_cat = f1_score(y_test_val["bottom_category_id"], y_val["bottom_category_id"], average="micro")
f1_color = f1_score(y_test_val["color_id"], y_val["color_id"], average="micro")

In [32]:
print("F1 score for top category: {:.3f}".format(f1_top_cat))
print("F1 score for bottom category: {:.3f}".format(f1_bottom_cat))
print("F1 score for color: {:.3f}".format(f1_color))

F1 score for top category: 1.000
F1 score for bottom category: 1.000
F1 score for color: 1.000
